In [ ]:
import os
import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot
from keras.preprocessing import image
from scipy.misc import toimage
from random import shuffle
import cv2
from keras.preprocessing.image import ImageDataGenerator #for data augmentation
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as k
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, Callback
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

In [ ]:
train_path = '/home/prateeksha/Downloads/kaggle/catsvsdogs/train'
train_labels = os.listdir(train_path)
train_labels[1]
len(train_labels)

In [ ]:
for i in range(0, 9):
    label = train_labels[i]
    cur_path = train_path + "/" + label
    print cur_path
    img = image.load_img(cur_path, target_size=(224,224))
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(toimage(img))
# show the plot
pyplot.show()

In [ ]:
y_train = []
for i,img in enumerate(train_labels):
    split = img.split(".")
    split = split[0]
    #print split
    if split == 'cat': num = 1
    #                             [no cat, very doggo]
    elif split == 'dog': num=0
    y_train.append(num)
len(y_train)

In [ ]:
x_train = []
for i,img in enumerate(train_labels):
    path = os.path.join(train_path, img)
    img = image.load_img(path, target_size = (224,224))
    img = img_to_array(img)
    img=np.array(img) 
    img=preprocess_input(img)
    x_train.append(img)
    if i%100 == 0: print('Processed {} of {}'.format(i, 1000))
    
print(x_train[0].shape)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(x_train, y_train, test_size = 0.25, random_state = 42)


In [ ]:
img_rows, img_cols = 224, 224
if k.image_data_format() == 'channels_first':
    trainX = trainX.reshape(trainX.shape[0], 3, img_rows, img_cols)
    testX = testX.reshape(testX.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    trainX = trainX.reshape(trainX.shape[0], img_rows, img_cols, 3)
    testX = testX.reshape(testX.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [ ]:
x_train = np.array(x_train, dtype="float")/255.0
y_train = np.array(y_train)
print x_train.shape


#we need to hot-encode the labels  
trainY = keras.utils.to_categorical(trainY, num_classes = 2)
testY = keras.utils.to_categorical(testY, num_classes =2)


# Using VGG-16 (freezing the initial layers and fine-tuning the last layers)

In [ ]:
from keras.models import Model

def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):

    model = VGG16(weights='imagenet', include_top=True)

    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    model.layers[-1].outbound_nodes = []

    x=Dense(num_classes, activation='softmax')(model.output)

    model=Model(model.input,x)

#To set the first 8 layers to non-trainable (weights will not be updated)

    for layer in model.layers[:8]:
        layer.trainable = False


    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
img_rows, img_cols = 224,224 # Resolution of inputs
channel = 3
num_classes = 2 
batch_size = 16 
nb_epoch = 10
model = vgg16_model(img_rows, img_cols, channel, num_classes)

In [ ]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

#early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')        
        
def run_catdog():
    history = LossHistory()
    model.fit(trainX, trainY, batch_size = batch_size, validation_data = (testX, testY), callbacks= [history], epochs = 10, verbose =1, shuffle =True)
   
   

    #predictions = model.predict(test, verbose=0)
    return  history
history = run_catdog()

In [ ]:
loss = history.losses
val_loss = history.val_losses

pyplot.xlabel('Epochs')
pyplot.ylabel('Loss')
pyplot.title('Loss Trend')
pyplot.plot(loss, 'blue', label='Training Loss')
pyplot.plot(val_loss, 'green', label='Validation Loss')
pyplot.xticks(range(0,10)[0::2])
pyplot.legend()
pyplot.show()